In [ ]:


from google.colab import files
files.upload()  # завантаж kaggle.json


!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d moazeldsokyx/dogs-vs-cats
!unzip -q dogs-vs-cats.zip -d data


import os, shutil
from sklearn.model_selection import train_test_split

cat_dir = "data/training_set/training_set/cats"
dog_dir = "data/training_set/training_set/dogs"

cat_images = [os.path.join(cat_dir, f) for f in os.listdir(cat_dir)]
dog_images = [os.path.join(dog_dir, f) for f in os.listdir(dog_dir)]

cat_train, cat_val = train_test_split(cat_images, test_size=0.2, random_state=42)
dog_train, dog_val = train_test_split(dog_images, test_size=0.2, random_state=42)

def make_dirs(base):
    for subset in ['train', 'val']:
        for cls in ['cats', 'dogs']:
            os.makedirs(os.path.join(base, subset, cls), exist_ok=True)

def copy_imgs(filelist, target_dir):
    for filepath in filelist:
        shutil.copy(filepath, target_dir)

make_dirs('dataset')
copy_imgs(cat_train, 'dataset/train/cats')
copy_imgs(cat_val, 'dataset/val/cats')
copy_imgs(dog_train, 'dataset/train/dogs')
copy_imgs(dog_val, 'dataset/val/dogs')


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    'dataset/train', target_size=(150,150), batch_size=32, class_mode='binary')
val_data = val_gen.flow_from_directory(
    'dataset/val', target_size=(150,150), batch_size=32, class_mode='binary')

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, epochs=10, validation_data=val_data)


plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Точність класифікації')
plt.legend()
plt.grid()
plt.show()


model.save('cats_vs_dogs_model.h5')